In [1]:
import matplotlib.pyplot as plt

In [2]:
import pandas as pd
from pathlib import Path

def imgPaths(filepaths, test_set=False):
    labels = []
    filepaths_str = []  # List to store the string representations of filepaths
    
    for filepath in filepaths:
        # Ensure filepath is a string
        if isinstance(filepath, Path):
            filepath_str = str(filepath)
        elif isinstance(filepath, str):
            filepath_str = filepath
        else:
            raise ValueError("Filepath must be a string or a Path object")

        filepaths_str.append(filepath_str)

        # Use Path methods to handle path separators properly
        path_parts = Path(filepath_str).parts

        if test_set:
            # Handle test set paths
            # Extract the label from the file name or a specific part of the path
            try:
                filename = Path(filepath_str).stem
                label = filename.split('_')[0]  # Assuming labels are prefixed before underscores
            except IndexError:
                label = 'unknown'
        else:
            # Handle training set paths
            try:
                label = path_parts[-2]  # Extract the label (second-to-last segment in the path)
            except IndexError:
                label = 'unknown'

        labels.append(label)
    
    # Create DataFrame
    df = pd.DataFrame({
        'Filepath': filepaths_str,
        'Label': labels
    })

    # Shuffle the DataFrame and reset index
    df = df.sample(frac=1).reset_index(drop=True)
    
    return df

# Create Variables
train_image_dir = Path('data/asl_alphabet/asl_alphabet_train')
train_filepaths = list(train_image_dir.glob(r'**/*.jpg'))

test_image_dir = Path('data/asl_alphabet/asl_alphabet_test')
test_filepaths = list(test_image_dir.glob(r'**/*.jpg'))

# Create DataFrames
train_df = imgPaths(train_filepaths)
test_df = imgPaths(test_filepaths, test_set=True)

# Display DataFrames
print(train_df.head())
print(train_df.shape)
print(test_df.head())
print(test_df.shape)


                                            Filepath  Label
0  data/asl_alphabet/asl_alphabet_train/space/spa...  space
1    data/asl_alphabet/asl_alphabet_train/N/N197.jpg      N
2    data/asl_alphabet/asl_alphabet_train/C/C195.jpg      C
3   data/asl_alphabet/asl_alphabet_train/O/O1379.jpg      O
4  data/asl_alphabet/asl_alphabet_train/del/del24...    del
(87000, 2)
                                         Filepath Label
0  data/asl_alphabet/asl_alphabet_test/F_test.jpg     F
1  data/asl_alphabet/asl_alphabet_test/J_test.jpg     J
2  data/asl_alphabet/asl_alphabet_test/T_test.jpg     T
3  data/asl_alphabet/asl_alphabet_test/U_test.jpg     U
4  data/asl_alphabet/asl_alphabet_test/C_test.jpg     C
(29, 2)


In [3]:
train_df['Label']

0        space
1            N
2            C
3            O
4          del
         ...  
86995      del
86996        E
86997        C
86998        M
86999        G
Name: Label, Length: 87000, dtype: object

In [4]:
train_df['Label'].unique()

array(['space', 'N', 'C', 'O', 'del', 'Z', 'H', 'U', 'X', 'S', 'G', 'L',
       'E', 'M', 'P', 'J', 'W', 'R', 'Y', 'Q', 'V', 'I', 'B', 'K',
       'nothing', 'F', 'A', 'T', 'D'], dtype=object)

In [ ]:
import tensorflow as tf

train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2,
    horizontal_flip = True,
    brightness_range=(0.75, 1.3),
    rotation_range=20,
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2,
    horizontal_flip = True,
    brightness_range=(0.75, 1.3),
    rotation_range=20,
)

train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

In [ ]:
# reuse all the layers except for the output layer: include_top=False.
pretrainedModel = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg' # reduces each feature map to a single value by averaging,
)

# freeze the reused layers during the first few epochs.
pretrainedModel.trainable = False

# input layer is same.
inputs = pretrainedModel.input

# output layer: add custom layers on top.
x = tf.keras.layers.Dense(128, activation='relu')(pretrainedModel.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)
outputs = tf.keras.layers.Dense(29, activation='softmax')(x)

# create the final model.
model = tf.keras.Model(inputs=inputs, outputs=outputs)

# since the new output layer was initialized randomly it will make large errors (at least during the first few epochs), so
# there will be large error gradients that may wreck the reused weights. To avoid this,
# one approach is to freeze the reused layers during the first few epochs, 
# giving the new layer some time to learn reasonable weights.
# default learning rate (LR) of the Adam optimizer in TensorFlow/Keras is 0.001.
adam = tf.keras.optimizers.Adam(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam'
)

# always compile your model after you freeze or unfreeze layers.
model.compile(
    optimizer=adam,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Print model summary
model.summary()

In [ ]:
# train the model for a few epochs.
history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=2,
    callbacks=[
         tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=2,
            restore_best_weights=True
         )
     ]
)

In [ ]:
# unfreeze the reused layers.
for layer in model.layers[:-3]:
    layer.trainable = True

# after unfreezing the reused layers, it is usually a good idea to reduce the learning rate.
adam = tf.keras.optimizers.Adam(
    learning_rate=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam'
)

# compiling the model again.
model.compile(
    optimizer=adam,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# continue training to fine-tune the reused layers.
history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=8,
    callbacks=[
         tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=2,
            restore_best_weights=True
         )
     ]
)

In [ ]:
model.save('asl_alphabet_classifier.h5')
results = model.evaluate(test_images)
print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

# DRAW Results
%matplotlib inline
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
# https://www.kaggle.com/code/thesnak/asl-classfier

In [28]:
import tensorflow as tf
from tensorflow import keras
from keras import saving

In [30]:
load_model = saving.load_model('asl_alphabet_classifier.keras')

/Users/hanhf/anaconda3/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'Adam', because it has 326 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [33]:
load_model.export('models/asl_alphabet_classifier')

INFO:tensorflow:Assets written to: models/asl_alphabet_classifier/assets


INFO:tensorflow:Assets written to: models/asl_alphabet_classifier/assets


Saved artifact at 'models/asl_alphabet_classifier'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 29), dtype=tf.float32, name=None)
Captures:
  14277577616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14277578960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14277579344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14277578768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14277578192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14277579920: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14277581264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14277581456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14277581072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14277580304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14277581840: TensorSpec(shape=(), dtype=

In [34]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model('models/asl_alphabet_classifier') # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

W0000 00:00:1724153815.578997  300851 tf_tfl_flatbuffer_helpers.cc:390] Ignored output_format.
W0000 00:00:1724153815.579761  300851 tf_tfl_flatbuffer_helpers.cc:393] Ignored drop_control_dependency.
